In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_989064/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        # self.discard_below_average(sort_by="r2")
        # self.discard_below_average(sort_by="r2_vt")
        # self.discard_high_standard_deviation()
        self.results_df.sort_values(by="mse")
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.4)
analize.Analize()
# analize.clean_folder(subfolder="dataset", extension="pkl")
# analize.clean_folder(subfolder="results", extension="xlsx")
# analize.clean_folder(subfolder="results", extension="txt")
# analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_1_9_4,0.999886,0.827195,0.818958,0.999871,0.000492,1.155549,0.647578,0.000940,0.041825,0.022186,1.007276,0.022536,145.233137,286.568310,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
1,model_1_9_6,0.999880,0.827450,0.818373,0.999834,0.000521,1.153840,0.649670,0.001212,0.044763,0.022828,1.007703,0.023188,145.119025,286.454198,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
2,model_1_9_7,0.999879,0.827388,0.818364,0.999829,0.000522,1.154257,0.649700,0.001248,0.044936,0.022850,1.007718,0.023210,145.115240,286.450412,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
3,model_1_9_5,0.999869,0.827326,0.818244,0.999802,0.000568,1.154670,0.650129,0.001441,0.047806,0.023842,1.008402,0.024218,144.945219,286.280392,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
4,model_1_9_3,0.999865,0.827268,0.818466,0.999917,0.000583,1.155061,0.649338,0.000604,0.046409,0.024151,1.008622,0.024532,144.893691,286.228864,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
478,model_1_2_1,0.952637,0.750910,0.776561,0.949758,0.205069,1.665666,0.799228,0.153397,0.510693,0.452845,4.031214,0.459977,133.168820,274.503993,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
479,model_1_2_0,0.943212,0.741556,0.762527,0.939862,0.245879,1.728217,0.849428,0.183610,0.617552,0.495862,4.634455,0.503672,132.805828,274.141001,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
480,model_1_1_2,0.922385,0.721136,0.741306,0.896214,0.336053,1.864762,0.925334,0.570732,0.762367,0.579701,5.967348,0.588830,132.180974,273.516147,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
481,model_1_1_1,0.904965,0.705084,0.716364,0.884174,0.411477,1.972105,1.014550,0.636938,0.942431,0.641465,7.082226,0.651567,131.776005,273.111178,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
